In [1]:
import torch
import os
import json
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
import numpy as np
from maestro.trainer.models.florence_2.core import train
from maestro.trainer.models.florence_2.inference import predict
from maestro.trainer.models.florence_2.checkpoints import OptimizationStrategy, load_model
import supervision as sv
from supervision.geometry.core import Position
from glob import glob
import shutil
from IPython.display import Image
from maestro.trainer.models.florence_2.checkpoints import OptimizationStrategy, load_model
import matplotlib.pyplot as plt
from maestro.trainer.common.datasets.coco import COCODataset
from maestro.trainer.models.florence_2.loaders import evaluation_collate_fn, train_collate_fn
from maestro.trainer.models.florence_2.detection import (
    detections_to_prefix_formatter,
    detections_to_suffix_formatter,
)
from functools import partial
from maestro.trainer.common.datasets.core import create_data_loaders
from maestro.trainer.models.florence_2.inference import predict_with_inputs
from tqdm import tqdm
import pandas as pd
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
id = 34

In [3]:
from maestro.trainer.models.florence_2.checkpoints import OptimizationStrategy, load_model

processor, model = load_model(
    model_id_or_path=f"/home/nupoor.assudani/cs299_main/florence2_finetuning/training/florence_2/{id}/checkpoints/latest",
    revision = "refs/heads/main",
    # optimization_strategy=OptimizationStrategy.NONE,
)

Error during conversion: ChunkedEncodingError(ProtocolError('Response ended prematurely'))
Florence2LanguageForConditionalGeneration has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


In [4]:
from maestro.trainer.common.datasets.coco import COCODataset

test_dataset = COCODataset(
    annotations_path="./dynamic_lucknow_coco_train_test/train/_annotations.coco.json",
    images_directory_path="./dynamic_lucknow_coco_train_test/train",
    # annotations_path="/home/shataxi.dubey/shataxi_work/vlm_on_planet/Florence-2/poker-cards-4/test/_annotations.coco.json",
    # images_directory_path="/home/shataxi.dubey/shataxi_work/vlm_on_planet/Florence-2/poker-cards-4/test"
)

CLASSES = test_dataset.classes
CLASSES

[INFO] maestro - Loaded 32 valid entries out of 32 from './dynamic_lucknow_coco_train_test/train/_annotations.coco.json'. Skipped 0. Found 0 entries with empty detections.


['brick kilns with chimney', 'background']

In [5]:
test_dataset.entries

[('./dynamic_lucknow_coco_train_test/train/8978754_3076282.tif',
  Detections(xyxy=array([[ 21, 122,  47, 153]]), mask=None, confidence=None, class_id=array([0]), tracker_id=None, data={}, metadata={})),
 ('./dynamic_lucknow_coco_train_test/train/8978754_3077505.tif',
  Detections(xyxy=array([[  0,   0, 320, 320]]), mask=None, confidence=None, class_id=array([1]), tracker_id=None, data={}, metadata={})),
 ('./dynamic_lucknow_coco_train_test/train/8978754_3078728.tif',
  Detections(xyxy=array([[  0,   0, 320, 320]]), mask=None, confidence=None, class_id=array([1]), tracker_id=None, data={}, metadata={})),
 ('./dynamic_lucknow_coco_train_test/train/8978754_3079951.tif',
  Detections(xyxy=array([[  0,   0, 320, 320]]), mask=None, confidence=None, class_id=array([1]), tracker_id=None, data={}, metadata={})),
 ('./dynamic_lucknow_coco_train_test/train/8978754_3081174.tif',
  Detections(xyxy=array([[  0,   0, 320, 320]]), mask=None, confidence=None, class_id=array([1]), tracker_id=None, data

In [6]:
from maestro.trainer.models.florence_2.loaders import evaluation_collate_fn, train_collate_fn
from maestro.trainer.models.florence_2.detection import (
    detections_to_prefix_formatter,
    detections_to_suffix_formatter,
)

from functools import partial

from maestro.trainer.common.datasets.core import create_data_loaders

train_loader, valid_loader, test_loader = create_data_loaders(
                    dataset_location= './dynamic_lucknow_coco_train_test',
                    train_batch_size= 32,
                    train_collect_fn= partial(train_collate_fn, processor=processor),
                    # train_num_workers=10,
                    test_batch_size= 4,
                    test_collect_fn= partial(evaluation_collate_fn, processor=processor),
                    detections_to_prefix_formatter=detections_to_prefix_formatter,
                    detections_to_suffix_formatter=detections_to_suffix_formatter,
                    )

[INFO] maestro - Creating data loaders from './dynamic_lucknow_coco_train_test'...
[INFO] maestro - Detected COCO dataset format.
[INFO] maestro - Found COCO split 'train' at: ./dynamic_lucknow_coco_train_test/train/_annotations.coco.json
[INFO] maestro - Loaded 32 valid entries out of 32 from './dynamic_lucknow_coco_train_test/train/_annotations.coco.json'. Skipped 0. Found 0 entries with empty detections.
[INFO] maestro - Found COCO split 'valid' at: ./dynamic_lucknow_coco_train_test/valid/_annotations.coco.json
[INFO] maestro - Loaded 649 valid entries out of 649 from './dynamic_lucknow_coco_train_test/valid/_annotations.coco.json'. Skipped 0. Found 0 entries with empty detections.
[INFO] maestro - Found COCO split 'test' at: ./dynamic_lucknow_coco_train_test/test/_annotations.coco.json
[INFO] maestro - Loaded 649 valid entries out of 649 from './dynamic_lucknow_coco_train_test/test/_annotations.coco.json'. Skipped 0. Found 0 entries with empty detections.
[INFO] maestro - Initializ

In [7]:
CLASSES = test_loader.dataset.coco_dataset.classes
CLASSES

['brick kilns with chimney', 'background']

In [24]:
def add_class_ids_and_confidence(detection):

    detection.class_id = []
    detection.confidence = []
    if detection.data=={}:
        return None
    else:
        for class_name in detection.data['class_name']:
            try:
                detection.class_id.append(CLASSES.index(class_name))
            except:
                detection.class_id.append(len(CLASSES)) # if class name is different from ['brick kilns with chimney', 'background'] assign class id other than 0 & 1
            detection.confidence.append(1) # florence_2 does not give confidence score so no use of conf_threshold
        
    return detection

In [25]:
import supervision as sv

from maestro.trainer.models.florence_2.inference import predict_with_inputs

predictions = []
targets = []

for input_ids, pixel_values, images, prefixes, suffixes in tqdm(test_loader):
    generated_texts = predict_with_inputs(model=model, processor=processor, input_ids=input_ids, pixel_values=pixel_values, device = model.device)

    for generated_text in generated_texts:
        predicted_result = processor.post_process_generation(text=generated_text, task="<OD>", image_size=(images[0].width, images[0].height))
        print(predicted_result)
        predicted_result = sv.Detections.from_vlm(vlm='florence_2',
                result=predicted_result,
                resolution_wh=(320, 320))
        predicted_result = add_class_ids_and_confidence(predicted_result)
        predictions.append(predicted_result)

    for suffix in suffixes:
        target_result = processor.post_process_generation(text=suffix, task="<OD>", image_size=(images[0].width, images[0].height))
        target_result = sv.Detections.from_vlm(vlm='florence_2',
                result=target_result,
                resolution_wh=(320, 320))
        print(target_result)
        target_result = add_class_ids_and_confidence(target_result)
        targets.append(target_result)
    

  1%|          | 1/163 [00:02<07:08,  2.65s/it]

{'<OD>': {'bboxes': [], 'labels': []}}
{'<OD>': {'bboxes': [], 'labels': []}}
{'<OD>': {'bboxes': [], 'labels': []}}
{'<OD>': {'bboxes': [], 'labels': []}}
Detections(xyxy=array([[1.6000e-01, 1.6000e-01, 3.2016e+02, 3.2016e+02]], dtype=float32), mask=None, confidence=None, class_id=None, tracker_id=None, data={'class_name': array(['background'], dtype='<U10')}, metadata={})
Detections(xyxy=array([[1.6000e-01, 1.6000e-01, 3.2016e+02, 3.2016e+02]], dtype=float32), mask=None, confidence=None, class_id=None, tracker_id=None, data={'class_name': array(['background'], dtype='<U10')}, metadata={})
Detections(xyxy=array([[1.6000e-01, 1.6000e-01, 3.2016e+02, 3.2016e+02]], dtype=float32), mask=None, confidence=None, class_id=None, tracker_id=None, data={'class_name': array(['background'], dtype='<U10')}, metadata={})
Detections(xyxy=array([[1.6000e-01, 1.6000e-01, 3.2016e+02, 3.2016e+02]], dtype=float32), mask=None, confidence=None, class_id=None, tracker_id=None, data={'class_name': array(['bac

  1%|          | 2/163 [00:05<07:15,  2.71s/it]

{'<OD>': {'bboxes': [], 'labels': []}}
{'<OD>': {'bboxes': [[0.1599999964237213, 0.1599999964237213, 319.5199890136719, 319.5199890136719]], 'labels': ['clock tower']}}
{'<OD>': {'bboxes': [], 'labels': []}}
{'<OD>': {'bboxes': [], 'labels': []}}
Detections(xyxy=array([[1.6000e-01, 1.6000e-01, 3.2016e+02, 3.2016e+02]], dtype=float32), mask=None, confidence=None, class_id=None, tracker_id=None, data={'class_name': array(['background'], dtype='<U10')}, metadata={})
Detections(xyxy=array([[1.6000e-01, 1.6000e-01, 3.2016e+02, 3.2016e+02]], dtype=float32), mask=None, confidence=None, class_id=None, tracker_id=None, data={'class_name': array(['background'], dtype='<U10')}, metadata={})
Detections(xyxy=array([[1.6000e-01, 1.6000e-01, 3.2016e+02, 3.2016e+02]], dtype=float32), mask=None, confidence=None, class_id=None, tracker_id=None, data={'class_name': array(['background'], dtype='<U10')}, metadata={})
Detections(xyxy=array([[1.6000e-01, 1.6000e-01, 3.2016e+02, 3.2016e+02]], dtype=float32), 

  2%|▏         | 3/163 [00:06<05:10,  1.94s/it]

...

{'<OD>': {'bboxes': [], 'labels': []}}
{'<OD>': {'bboxes': [[0.1599999964237213, 41.7599983215332, 21.279998779296875, 63.19999694824219]], 'labels': ['brick kilns with chimney']}}
{'<OD>': {'bboxes': [[149.9199981689453, 39.20000076293945, 209.1199951171875, 80.15999603271484]], 'labels': ['brick kilns with chimney']}}
{'<OD>': {'bboxes': [[0.1599999964237213, 0.1599999964237213, 319.5199890136719, 319.5199890136719]], 'labels': ['maple']}}
Detections(xyxy=array([[1.6000e-01, 1.6000e-01, 3.2016e+02, 3.2016e+02]], dtype=float32), mask=None, confidence=None, class_id=None, tracker_id=None, data={'class_name': array(['background'], dtype='<U10')}, metadata={})
Detections(xyxy=array([[126.24    ,  81.119995, 156.31999 , 107.04    ]], dtype=float32), mask=None, confidence=None, class_id=None, tracker_id=None, data={'class_name': array(['brick kilns with chimney'], dtype='<U24')}, metadata={})
Detections(xyxy=array([[1.6000e-01, 1.6000e-01, 3.2016e+02, 3.2016e+02]], dtype=float32), mask=Non

 99%|█████████▉| 162/163 [02:02<00:00,  1.13it/s]

{'<OD>': {'bboxes': [[0.1599999964237213, 0.1599999964237213, 319.5199890136719, 319.5199890136719]], 'labels': ['maple']}}
{'<OD>': {'bboxes': [[300.0, 297.1199951171875, 315.03997802734375, 318.239990234375]], 'labels': ['brick kilns with chimney']}}
{'<OD>': {'bboxes': [], 'labels': []}}
{'<OD>': {'bboxes': [], 'labels': []}}
Detections(xyxy=array([[1.6000e-01, 1.6000e-01, 3.2016e+02, 3.2016e+02]], dtype=float32), mask=None, confidence=None, class_id=None, tracker_id=None, data={'class_name': array(['background'], dtype='<U10')}, metadata={})
Detections(xyxy=array([[1.6000e-01, 1.6000e-01, 3.2016e+02, 3.2016e+02]], dtype=float32), mask=None, confidence=None, class_id=None, tracker_id=None, data={'class_name': array(['background'], dtype='<U10')}, metadata={})
Detections(xyxy=array([[166.23999, 253.28   , 193.12   , 280.16   ],
       [ 15.2    , 238.23999,  40.16   , 264.16   ]], dtype=float32), mask=None, confidence=None, class_id=None, tracker_id=None, data={'class_name': array(['

100%|██████████| 163/163 [02:03<00:00,  1.32it/s]

{'<OD>': {'bboxes': [[0.1599999964237213, 0.1599999964237213, 319.5199890136719, 319.5199890136719]], 'labels': ['brick chimney']}}
Detections(xyxy=array([[1.6000e-01, 1.6000e-01, 3.2016e+02, 3.2016e+02]], dtype=float32), mask=None, confidence=None, class_id=None, tracker_id=None, data={'class_name': array(['background'], dtype='<U10')}, metadata={})


In [18]:
print(predicted_result.data)

{}


In [26]:
df = pd.DataFrame({}, columns = ['IoU', 'Precision', 'Recall', 'F1 score', 'TP', 'FP', 'FN', 'Kiln instances'])
for iou in [0.1,0.3,0.5,0.7]:
    confusion_matrix = sv.ConfusionMatrix.from_detections(
        predictions=predictions,
        targets=targets,
        classes=CLASSES,
        conf_threshold = 0.25, # florence_2 does not give confidence score so no use of conf_threshold  
        iou_threshold=iou
    )

    # calculate precision recall and f1-score
    cm = confusion_matrix.matrix
    tp = cm[0][0]
    predicted_positives = cm[:,0].sum()
    actual_positives = cm[0, :].sum()
    precision = tp/ (predicted_positives + 1e-9)
    recall = tp/ (actual_positives + 1e-9)
    f1_score = 2*precision*recall / (precision + recall)
    false_positives = predicted_positives - tp

    df = pd.concat([df, pd.DataFrame({'IoU': iou, 'Precision': precision, 'Recall': recall, 'F1 score': f1_score, 'TP': tp, 'FP': false_positives, 'FN': actual_positives - tp, 'Kiln instances': actual_positives}, index = [0])])
display(df)

print(f'\n\nPlot of Confusion matrix at IoU {iou}')
_ = confusion_matrix.plot()

AttributeError: 'NoneType' object has no attribute 'class_id'